In [1]:
# Fix dependencies by pinning numpy to <2.0 and reinstalling critical packages
import subprocess
import sys

def install_packages():
    packages = [
        "numpy<2.0",           # Downgrade numpy to 1.x to fix SciPy/TensorFlow compatibility
        "scipy",               # Ensure SciPy is consistent with the numpy version
        "open3d",
        "opencv-python",
        "plyfile",
        "gsplat"
    ]
    
    # Construct the pip install command
    # Added --no-cache-dir to prevent issues with cached broken builds
    command = [sys.executable, "-m", "pip", "install"] + packages + ["--upgrade", "--no-cache-dir"]
    
    print("Fixing dependencies... This may take a minute.")
    try:
        subprocess.check_call(command)
        print("Successfully fixed dependencies.")
    except subprocess.CalledProcessError as e:
        print(f"Error during installation: {e}")

install_packages()

Fixing dependencies... This may take a minute.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
Successfully fixed dependencies.


In [2]:
import numpy as np
import os
import time
import scipy
import scipy.ndimage
import heapq
import math
import sys
import subprocess
from scipy.ndimage import map_coordinates, label
from scipy.interpolate import splprep, splev, interp1d
from scipy.spatial.distance import cdist
from scipy.spatial.transform import Rotation as R

print("Starting initialization and imports...")

# ====================================================================================
# 0. Dependencies Imports and Checks
# ====================================================================================

try:
    import torch
    TORCH_AVAILABLE = True
except ImportError:
    print("WARNING: Torch not available.")
    TORCH_AVAILABLE = False
    torch = None

try:
    import open3d as o3d
except ImportError:
    print("ERROR: Open3D not found. Ensure installation cell ran successfully.")
    o3d = None

try:
    from plyfile import PlyData
except ImportError:
    print("ERROR: Plyfile not found.")
    PlyData = None

try:
    import matplotlib.pyplot as plt
except ImportError:
    plt = None

try:
    import cv2
except ImportError:
    cv2 = None

GSPLAT_AVAILABLE = False
if TORCH_AVAILABLE:
    try:
        import gsplat
        GSPLAT_AVAILABLE = True
    except ImportError:
        print("INFO: gsplat not found.")
    except Exception as e:
        print(f"INFO: gsplat import issue: {e}.")

Starting initialization and imports...
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:
# ====================================================================================
# 1. Global Configuration (UPDATED)
# ====================================================================================
class Config:
    # --- General Parameters ---
    # Probably will need to change paths
    INPUT_PLY_PATH = "/kaggle/input/ply-scenes/input-data/outdoor-drone_exported.ply"
    OUTPUT_DIR = "./output_results"
    JSON_PATH = "/kaggle/input/drone-path-v3/planned_path.json" 

    # Determine primary compute device
    DEVICE = None
    if TORCH_AVAILABLE and torch:
        if torch.cuda.is_available():
            DEVICE = torch.device("cuda:0")
        else:
            DEVICE = torch.device("cpu")

    # --- Stage 1 Parameters (Geometry) ---
    OPACITY_THRESHOLD = 0.05
    SOR_NB_NEIGHBORS = 30
    SOR_STD_RATIO = 2.5
    TARGET_VOXEL_RESOLUTION = 300

    # --- Stage 2 Parameters (Navigation Field) (UPDATED) ---
    CLOSING_RADIUS_VOXELS = 2
    # FIX: Adjusted safety radii for better indoor navigation balance.
    R_SAFE_PLANNING_METERS = 0.40  # (was 0.8)
    R_SAFE_MINIMUM_METERS = 0.15

    # --- Stage 3 Parameters (Coverage) ---
    TARGET_COVERAGE_RATIO = 0.85
    MAX_WAYPOINTS = 12
    CANDIDATE_SAMPLING_STRIDE = 5 
    TARGET_DOWNSAMPLE_VOXEL_FACTOR = 3.5
    RAY_OFFSET = 0.01

    # --- Stage 4 Parameters (Path Planning) ---
    TARGET_AVERAGE_SPEED = 1.2 # m/s
    VIDEO_FPS = 60
    INITIAL_SMOOTHING_FACTOR_RATIO = 2.0
    MIN_SMOOTHING_FACTOR_RATIO = 0.001
    SMOOTHING_REDUCTION_FACTOR = 0.5
    COLLISION_CHECK_DENSITY = 100

    # --- Stage 5 Parameters (Rendering) ---
    # Global Up Vector: This will be dynamically updated in Stage 1.6.
    GLOBAL_UP_VECTOR = np.array([0, 0, 1], dtype=np.float32) # Default fallback
    GIMBAL_SMOOTHING_SIGMA = 15.0
    RENDER_WIDTH = 1280
    RENDER_HEIGHT = 720
    FOV_Y_DEGREES = 60.0
    NEAR_PLANE = 0.01
    FAR_PLANE = 100.0

# Initialize O3D_DEVICE
Config.O3D_DEVICE = None
if o3d and hasattr(o3d, 'core'):
    try:
        if Config.DEVICE and "cuda" in str(Config.DEVICE):
             Config.O3D_DEVICE = o3d.core.Device("CUDA:0")
        else:
            Config.O3D_DEVICE = o3d.core.Device("CPU:0")
    except Exception:
        Config.O3D_DEVICE = o3d.core.Device("CPU:0")

In [4]:
# ====================================================================================
# 2. Global Variables (Inter-Stage Communication)
# ====================================================================================
STAGE1_PCD = None; STAGE1_VOXEL_GRID = None; STAGE1_VOXEL_SIZE = 0.0; STAGE1_SCENE_CENTER = np.array([0.0, 0.0, 0.0])
STAGE4_FINAL_TRAJECTORY = None; STAGE4_TIMESTAMPS = None
STAGE5_CAMERA_POSES_C2W_ORIGINAL = None


In [11]:
# ====================================================================================
# 3. Utility Functions
# ====================================================================================
def print_environment_info():
    print("--- Environment Info ---")
    if TORCH_AVAILABLE and torch:
        print(f"Compute Device: {Config.DEVICE}")
    if Config.O3D_DEVICE: print(f"Open3D Tensor Device: {Config.O3D_DEVICE}")
    print(f"GSplat Available: {GSPLAT_AVAILABLE}")
    print("------------------------")


In [12]:
# ====================================================================================
# 4. Stage 1: Loading and Geometry Analysis (UPDATED with Orientation Analysis)
# ====================================================================================

def load_and_filter_ply(ply_path, opacity_threshold):
    # (Implementation remains similar to original)
    print(f"\n--- Step 1.1: Loading PLY file from {ply_path} ---")
    if o3d is None or PlyData is None: return np.array([])
    if not os.path.exists(ply_path):
        print("ERROR: File not found."); return np.array([])

    start_time = time.time()
    try:
        plydata = PlyData.read(ply_path)
        vertex_data = plydata.elements[0]
        coordinates = np.stack((vertex_data['x'], vertex_data['y'], vertex_data['z']), axis=-1)

        if 'opacity' in vertex_data.data.dtype.names:
            opacity = np.asarray(vertex_data['opacity'])
            mask = opacity >= opacity_threshold
            filtered_coordinates = coordinates[mask]
        else:
            filtered_coordinates = coordinates

        print(f"Loaded {len(coordinates)} points. Filtered to {len(filtered_coordinates)}.")
        print(f"Step 1.1 completed in {time.time() - start_time:.2f} seconds.")
        return filtered_coordinates
    except Exception as e:
        print(f"ERROR: Failed to read PLY. Error: {e}."); return np.array([])

def create_and_clean_pcd(coordinates, sor_nb_neighbors, sor_std_ratio):
    # (Implementation remains similar to original)
    print("\n--- Step 1.2 & 1.3: Creating Open3D PCD and Cleaning (SOR) ---")
    if len(coordinates) == 0 or o3d is None: return None
    start_time = time.time()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coordinates)
    cleaned_pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=sor_nb_neighbors, std_ratio=sor_std_ratio)
    print(f"Points remaining: {len(cleaned_pcd.points)}")
    print(f"Step 1.2 & 1.3 completed in {time.time() - start_time:.2f} seconds.")
    return cleaned_pcd

def normalize_scene(pcd):
    # (Implementation remains similar to original)
    print("\n--- Step 1.4: Normalization (Centering) ---")
    if pcd is None: return None, np.array([0.0, 0.0, 0.0])
    center = pcd.get_center()
    pcd.translate(-center)
    return pcd, center

def adaptive_voxelization(pcd, target_resolution):
    # (Implementation remains similar to original)
    print("\n--- Step 1.5: Adaptive Voxelization ---")
    if pcd is None: return None, 0.0
    start_time = time.time()
    extent = pcd.get_axis_aligned_bounding_box().get_extent()
    longest_axis_length = np.max(extent)

    if longest_axis_length < 1e-6:
        voxel_size = 0.1
    else:
        voxel_size = longest_axis_length / target_resolution

    try:
        voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
    except Exception as e:
        print(f"ERROR: Voxelization failed: {e}"); return None, voxel_size

    print(f"Calculated adaptive voxel size: {voxel_size:.4f}")
    print(f"Step 1.5 completed in {time.time() - start_time:.2f} seconds.")
    return voxel_grid, voxel_size

def analyze_scene_orientation(pcd, voxel_size):
    """
    (NEW) Analyzes the point cloud normals to determine the dominant 'Up' vector automatically.
    """
    print("\n--- Step 1.6 (NEW): Analyzing Scene Orientation (Up-Vector) ---")
    if pcd is None or len(pcd.points) < 100:
        return Config.GLOBAL_UP_VECTOR

    start_time = time.time()

    # 1. Downsample and Estimate Normals
    analysis_pcd = pcd.voxel_down_sample(voxel_size=voxel_size * 1.5)
    radius = voxel_size * 3.0
    try:
        analysis_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30))
        analysis_pcd.orient_normals_consistent_tangent_plane(k=15)
    except Exception as e:
        print(f"WARNING: Normal estimation failed. Falling back to default Up-Vector.")
        return Config.GLOBAL_UP_VECTOR

    normals = np.asarray(analysis_pcd.normals)

    # 2. Analyze Normal Distribution (Histogram approach)
    BINS = 50
    hists = [np.histogram(normals[:, i], bins=BINS, range=(-1, 1))[0] for i in range(3)]

    # Score based on peaks at the extremes (floors/ceilings)
    peak_width = BINS // 10
    scores = [np.sum(h[:peak_width]) + np.sum(h[-peak_width:]) for h in hists]
    dominant_axis_idx = np.argmax(scores)

    # 3. Determine the Up direction (+ or -)
    up_vector = np.zeros(3)
    axis_names = ["X", "Y", "Z"]
    print(f"Dominant vertical axis detected: {axis_names[dominant_axis_idx]}")

    # Calculate the average direction of the dominant normals
    dominant_normals = normals[np.abs(normals[:, dominant_axis_idx]) > 0.8, dominant_axis_idx]
    mean_val = np.mean(dominant_normals) if len(dominant_normals) > 0 else 0

    if mean_val > 0.05:
        direction = 1
    elif mean_val < -0.05:
        direction = -1
    else:
        print("WARNING: Up direction is ambiguous. Falling back to default.")
        return Config.GLOBAL_UP_VECTOR

    up_vector[dominant_axis_idx] = direction

    print(f"Successfully Detected Global Up-Vector: {up_vector}")
    print(f"Step 1.6 completed in {time.time() - start_time:.2f} seconds.")
    return up_vector.astype(np.float32)

In [13]:
# ====================================================================================
# Stage 5: Cinematic Orientation and Rendering (UPDATED with GAF)
# ====================================================================================

def calculate_orientations_from_look_at(trajectory, look_at_targets, global_up_vector):
    """
    Calculate camera orientations based on look_at targets.
    The camera looks from each trajectory position towards the corresponding look_at target.
    Adopts OpenCV convention: X=Right, Y=Down, Z=Forward.
    """
    print("\n--- Step 5.1: Calculating Camera Orientations from Look-At Targets ---")
    start_time = time.time()
    
    N = len(trajectory)
    if N < 1:
        return np.array([[[1, 0, 0], [0, 1, 0], [0, 0, 1]]])
    
    if look_at_targets is None or len(look_at_targets) != N:
        print("WARNING: look_at_targets not provided or invalid, falling back to trajectory-based orientation")
        return calculate_base_orientations(trajectory, global_up_vector)
    
    # 1. Calculate Forward Vectors (Z-basis) from camera position to look_at target
    Z_basis = look_at_targets - trajectory
    
    # 2. Normalize forward vectors
    norms = np.linalg.norm(Z_basis, axis=1)
    
    # Handle cases where camera position equals look_at target
    for i in range(N):
        if norms[i] < 1e-6:
            # Use trajectory tangent as fallback
            if i < N - 1:
                Z_basis[i] = trajectory[i + 1] - trajectory[i]
            elif i > 0:
                Z_basis[i] = trajectory[i] - trajectory[i - 1]
            else:
                Z_basis[i] = np.array([0, 0, 1], dtype=np.float32)
            norms[i] = np.linalg.norm(Z_basis[i])
    
    # Normalize
    valid_norms = norms > 1e-6
    Z_basis[valid_norms] = Z_basis[valid_norms] / norms[valid_norms, np.newaxis]
    
    # 3. Calculate Right Vectors (X-basis) using Global Up
    X_basis = np.cross(global_up_vector, Z_basis)
    X_norms = np.linalg.norm(X_basis, axis=1)
    
    # 4. Handle Gimbal Lock
    locked = X_norms < 1e-3
    if np.any(locked):
        for i in range(N):
            if locked[i]:
                if i > 0:
                    X_basis[i] = X_basis[i-1]
                else:
                    arbitrary_vec = np.array([1, 0, 0], dtype=np.float32)
                    if abs(np.dot(arbitrary_vec, Z_basis[i])) > 0.99:
                        arbitrary_vec = np.array([0, 1, 0], dtype=np.float32)
                    X_basis[i] = np.cross(Z_basis[i], arbitrary_vec)
    
    # Normalize X_basis
    X_norms = np.linalg.norm(X_basis, axis=1)
    valid_X_norms = X_norms > 1e-6
    if np.all(valid_X_norms):
        X_basis = X_basis / X_norms[:, np.newaxis]
    else:
        X_basis[valid_X_norms] = X_basis[valid_X_norms] / X_norms[valid_X_norms, np.newaxis]
    
    # 5. Calculate True Up Vectors
    Up_basis = np.cross(Z_basis, X_basis)
    
    # 6. Ensure Up vectors point upward
    for i in range(N):
        if np.dot(Up_basis[i], global_up_vector) < 0:
            X_basis[i] = -X_basis[i]
            Up_basis[i] = -Up_basis[i]
    
    # 7. Enforce continuity
    for i in range(1, N):
        if np.dot(X_basis[i], X_basis[i-1]) < 0:
            X_basis[i] = -X_basis[i]
            Up_basis[i] = -Up_basis[i]
    
    # 8. Assemble Final Rotation Matrices (C2W) for OpenCV
    rotation_matrices = np.zeros((N, 3, 3))
    rotation_matrices[:, :, 0] = X_basis
    rotation_matrices[:, :, 1] = -Up_basis
    rotation_matrices[:, :, 2] = Z_basis
    
    print(f"Step 5.1 completed in {time.time() - start_time:.2f} seconds.")
    return rotation_matrices

def calculate_base_orientations(trajectory, global_up_vector):
    """
    (UPDATED) Step 5.1: Calculates orientations using Globally Aligned Frames (GAF).
    Replaces the previous Parallel Transport (RMF) method.
    Ensures the horizon remains level relative to the global_up_vector.
    Adopts OpenCV convention: X=Right, Y=Down, Z=Forward.
    """
    print("\n--- Step 5.1 (UPDATED): Calculating Globally Aligned Orientations (GAF) ---")
    start_time = time.time()

    N = len(trajectory)
    if N < 2:
        return np.array([[[1, 0, 0], [0, 1, 0], [0, 0, 1]]])

    # 1. Calculate Tangents (Forward Vectors = Z-basis)
    Z_basis = np.zeros((N, 3))
    # Central differences for smoothness
    Z_basis[1:-1] = trajectory[2:] - trajectory[:-2]
    Z_basis[0] = trajectory[1] - trajectory[0]
    Z_basis[-1] = trajectory[-1] - trajectory[-2]

    # 2. Normalize and handle zero velocity
    norms = np.linalg.norm(Z_basis, axis=1)
    # If the camera stops, maintain the previous direction
    for i in range(1, N):
        if norms[i] < 1e-6:
            Z_basis[i] = Z_basis[i-1]
            norms[i] = norms[i-1]

    if np.all(norms < 1e-6):
        return np.array([[[1, 0, 0], [0, 1, 0], [0, 0, 1]]] * N)

    # Apply normalization where norms are valid
    valid_norms = norms > 1e-6
    Z_basis[valid_norms] = Z_basis[valid_norms] / norms[valid_norms, np.newaxis]

    # 3. Calculate Right Vectors (X-basis) using Global Up Anchor
    # X = Cross(Global_Up, Z). This is the core of GAF.
    X_basis = np.cross(global_up_vector, Z_basis)
    X_norms = np.linalg.norm(X_basis, axis=1)

    # 4. Handle Gimbal Lock (Camera looking straight up or down)
    locked = X_norms < 1e-3
    if np.any(locked):
        # Where locked, reuse the X_basis from the previous frame for stability.
        for i in range(N):
            if locked[i]:
                if i > 0:
                    X_basis[i] = X_basis[i-1]
                else:
                    # Handle lock at the start: find an arbitrary orthogonal vector
                    arbitrary_vec = np.array([1, 0, 0], dtype=np.float32)
                    if abs(np.dot(arbitrary_vec, Z_basis[i])) > 0.99:
                            arbitrary_vec = np.array([0, 1, 0], dtype=np.float32)
                    # Cross product to find a valid X basis (ensure it's orthogonal to Z)
                    X_basis[i] = np.cross(Z_basis[i], arbitrary_vec)


    # Normalize X_basis again
    X_norms = np.linalg.norm(X_basis, axis=1)
    valid_X_norms = X_norms > 1e-6
    # Ensure division by zero is handled if any norms are still invalid
    if np.all(valid_X_norms):
         X_basis = X_basis / X_norms[:, np.newaxis]
    else:
         # Only normalize valid entries
         X_basis[valid_X_norms] = X_basis[valid_X_norms] / X_norms[valid_X_norms, np.newaxis]

    # 5. Calculate True Up Vectors
    # True_Up = Cross(Z, X) - ensures orthogonality
    Up_basis = np.cross(Z_basis, X_basis)
    
    # 6. Ensure Up vectors point generally upward (prevent flipping)
    # Check if Up vector aligns with global up, flip if pointing down
    for i in range(N):
        if np.dot(Up_basis[i], global_up_vector) < 0:
            # Flip both X and Up to maintain right-handed coordinate system
            X_basis[i] = -X_basis[i]
            Up_basis[i] = -Up_basis[i]
    
    # 7. Enforce continuity - prevent sudden flips between frames
    for i in range(1, N):
        # Check if X basis flipped relative to previous frame
        if np.dot(X_basis[i], X_basis[i-1]) < 0:
            X_basis[i] = -X_basis[i]
            Up_basis[i] = -Up_basis[i]

    # 8. Assemble Final Rotation Matrices (C2W) for OpenCV
    rotation_matrices = np.zeros((N, 3, 3))
    rotation_matrices[:, :, 0] = X_basis      # Right
    rotation_matrices[:, :, 1] = -Up_basis    # Down (OpenCV convention)
    rotation_matrices[:, :, 2] = Z_basis      # Forward

    print(f"Step 5.1 completed in {time.time() - start_time:.2f} seconds.")
    return rotation_matrices

def apply_camera_roll(rotation_matrices, roll_degrees):
    """
    Apply a roll rotation to camera orientations.
    Roll is rotation around the forward (Z) axis.
    Positive roll rotates the camera counter-clockwise when looking forward.
    
    Args:
        rotation_matrices: Nx3x3 array of rotation matrices
        roll_degrees: Roll angle in degrees (90 = rotate left, -90 = rotate right)
    
    Returns:
        Rotated camera matrices
    """
    print(f"\n--- Applying Camera Roll: {roll_degrees} degrees ---")
    
    # Create roll rotation matrix around Z axis
    roll_rad = np.deg2rad(roll_degrees)
    cos_r = np.cos(roll_rad)
    sin_r = np.sin(roll_rad)
    
    # Roll rotation matrix (rotation around Z axis)
    roll_matrix = np.array([
        [cos_r, -sin_r, 0],
        [sin_r, cos_r, 0],
        [0, 0, 1]
    ], dtype=np.float32)
    
    # Apply roll to each rotation matrix
    rolled_matrices = np.zeros_like(rotation_matrices)
    for i in range(len(rotation_matrices)):
        rolled_matrices[i] = rotation_matrices[i] @ roll_matrix
    
    return rolled_matrices

def smooth_orientations_gimbal(rotation_matrices, sigma):
    # (Implementation remains the same)
    print(f"\n--- Step 5.2 Smoothing (Sigma={sigma}) ---")
    N = len(rotation_matrices)
    if N < 2 or sigma <= 0: return rotation_matrices

    try:
        quats = R.from_matrix(rotation_matrices).as_quat()
    except: return rotation_matrices

    # Ensure quaternion continuity (prevent flips)
    for i in range(1, N):
        if np.dot(quats[i], quats[i-1]) < 0: quats[i] = -quats[i]

    # Apply Gaussian filter
    smoothed_q = scipy.ndimage.gaussian_filter1d(quats, sigma=sigma, axis=0, mode='reflect')
    # Normalize
    norms = np.linalg.norm(smoothed_q, axis=1, keepdims=True)
    smoothed_q /= (norms + 1e-8)
    return R.from_quat(smoothed_q).as_matrix()

def assemble_camera_poses(trajectory, rotation_matrices, scene_center):
    # (Implementation remains the same)
    print("\n--- 5.3 Assembling Poses ---")
    N = len(trajectory)
    poses_norm = np.eye(4)[np.newaxis].repeat(N, axis=0)
    poses_orig = np.eye(4)[np.newaxis].repeat(N, axis=0)
    
    poses_norm[:, :3, :3] = rotation_matrices; poses_norm[:, :3, 3] = trajectory
    # Denormalize position for original scene coordinates
    poses_orig[:, :3, :3] = rotation_matrices; poses_orig[:, :3, 3] = trajectory + scene_center
    return poses_norm, poses_orig

def visualize_final_path(pcd, final_traj, output_dir):
    # (Implementation simplified)
    print("\n--- 5.4 Saving Path Visualization ---")
    if o3d is None or pcd is None: return
    
    vis = o3d.geometry.PointCloud()
    try:
        # Downsample scene context
        scene_vis = pcd.voxel_down_sample(0.1)
        scene_vis.paint_uniform_color([0.7, 0.7, 0.7])
        vis += scene_vis
    except: pass
    
    if final_traj is not None:
        p = o3d.geometry.PointCloud()
        p.points = o3d.utility.Vector3dVector(final_traj)
        p.paint_uniform_color([0, 1, 0]) # Green path
        vis += p
        
    try:
        os.makedirs(output_dir, exist_ok=True)
        o3d.io.write_point_cloud(f"{output_dir}/path_vis.ply", vis)
        print(f"Saved {output_dir}/path_vis.ply")
    except: pass

# --- ROBUST RENDERING LOGIC ---
# (Implementation remains similar to original, robustified)

def load_full_ply_for_gsplat(path, device):
    print(f"Loading full PLY data for rendering from {path}...", flush=True)
    if not os.path.exists(path) or PlyData is None or torch is None: return None

    try:
        plydata = PlyData.read(path); v = plydata.elements[0]
    except: return None

    def to_ten(x):
        t = torch.tensor(x, dtype=torch.float32, device=device)
        if torch.isnan(t).any(): t = torch.nan_to_num(t)
        return t.contiguous()

    means = np.stack((v['x'], v['y'], v['z']), axis=-1); N = len(means)

    # Gracefully handle potentially missing fields
    try: opacities = v['opacity']
    except: opacities = np.ones((N, 1))
    try: scales = np.stack((v['scale_0'], v['scale_1'], v['scale_2']), axis=-1)
    except: scales = np.full((N, 3), 0.01)
    try: quats = np.stack((v['rot_0'], v['rot_1'], v['rot_2'], v['rot_3']), axis=-1)
    except: quats = np.zeros((N, 4)); quats[:, 0] = 1.0
    try: sh_dc = np.stack((v['f_dc_0'], v['f_dc_1'], v['f_dc_2']), axis=-1)
    except: sh_dc = np.full((N, 3), 0.5)

    # Handle SH Rest (Spherical Harmonics)
    extra_names = sorted([p.name for p in v.properties if p.name.startswith('f_rest_')], key=lambda x: int(x.split('_')[-1]))
    if len(extra_names) > 0:
        sh_rest = np.stack([v[n] for n in extra_names], axis=-1)
        if sh_rest.shape[1] % 3 == 0:
             sh_rest = sh_rest.reshape(N, sh_rest.shape[1] // 3, 3)
             shs = np.concatenate((sh_dc[:, None, :], sh_rest), axis=1)
        else: shs = sh_dc[:, None, :]
    else: shs = sh_dc[:, None, :]

    print(f"Loaded {N} gaussians. SH shape: {shs.shape}")
    return {'means': to_ten(means), 'scales': to_ten(scales), 'quats': to_ten(quats),
            'opacities': to_ten(opacities), 'shs': to_ten(shs)}


def render_video(poses, ply_path, config, output_dir):
    print("\n--- Step 5.5: Rendering Video (GSplat Rasterization) ---", flush=True)

    if cv2 is None or not GSPLAT_AVAILABLE or torch is None:
        print("Error: Missing dependencies."); return

    if not (torch.cuda.is_available() and config.DEVICE and "cuda" in str(config.DEVICE)):
        print("Warning: CUDA not available/configured. Rendering requires GPU."); return

    # 1. Load Data
    with torch.no_grad():
        g_data = load_full_ply_for_gsplat(ply_path, config.DEVICE)
    if g_data is None: return

    # 2. Determine SH Degree
    K = g_data['shs'].shape[1]
    try:
        sh_degree = int(math.sqrt(K) - 1)
        if (sh_degree + 1)**2 != K: sh_degree = 0
    except: sh_degree = 0
    print(f"Detected SH Degree: {sh_degree} (K={K})")

    # 3. Setup Video Writer
    H, W = config.RENDER_HEIGHT, config.RENDER_WIDTH
    os.makedirs(output_dir, exist_ok=True)
    video_path = os.path.join(output_dir, "cinematic_tour.mp4")

    try:
        writer = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), config.VIDEO_FPS, (W, H))
    except: print("ERROR: Initializing VideoWriter failed."); return

    # 4. Calculate Intrinsics
    fov_y_rad = np.deg2rad(config.FOV_Y_DEGREES)
    focal_length = H / (2.0 * np.tan(fov_y_rad / 2.0))
    K_mat = torch.tensor([[focal_length, 0, W/2.0], [0, focal_length, H/2.0], [0, 0, 1]], device=config.DEVICE, dtype=torch.float32)

    # Pre-process static attributes
    with torch.no_grad():
        means = g_data['means']
        scales = torch.exp(g_data['scales'])
        opacities = torch.sigmoid(g_data['opacities'])
        quats = g_data['quats']
        quats = quats / (quats.norm(dim=-1, keepdim=True) + 1e-8)
        shs = g_data['shs']

    # 5. Rendering Loop
    start_time = time.time()
    print(f"Rendering {len(poses)} frames...")
    for i, c2w in enumerate(poses):
        try:
            with torch.no_grad():
                c2w_torch = torch.tensor(c2w, dtype=torch.float32, device=config.DEVICE)
                w2c = torch.inverse(c2w_torch)

                render_output = gsplat.rasterization(
                    means=means, quats=quats, scales=scales, opacities=opacities, colors=shs,
                    viewmats=w2c.unsqueeze(0), Ks=K_mat.unsqueeze(0), width=W, height=H,
                    sh_degree=sh_degree, packed=False
                )

                if isinstance(render_output, tuple):
                    render_colors = render_output[0]
                else:
                    render_colors = render_output

                img_tensor = render_colors[0].clamp(0, 1)
                img_np = (img_tensor.cpu().numpy() * 255).astype(np.uint8)
                img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
                writer.write(img_bgr)

        except Exception as e:
            print(f"\nCRITICAL Render Error at frame {i}: {e}"); break

        if i % 50 == 0 or i == len(poses) - 1:
            elapsed = time.time() - start_time
            fps = (i + 1) / (elapsed + 1e-6)
            print(f"\rFrame {i+1}/{len(poses)}. Avg FPS: {fps:.2f}", end="", flush=True)

    print("\nRendering finished.")
    writer.release()

    # Cleanup GPU memory
    if torch:
        del g_data, means, scales, opacities, quats, shs
        if torch.cuda.is_available(): torch.cuda.empty_cache()

    print(f"Video saved: {video_path}")

In [22]:
# ====================================================================================
# 4. Main Pipeline Execution
# ====================================================================================

def load_path_from_json(json_path):
    """
    Load camera path from a JSON file.
    Expected format: list of dicts with 'pos' and 'look_at' keys.
    Returns trajectory positions and look_at targets (both Nx3 arrays).
    """
    import json
    print(f"\n--- Loading Path from {json_path} ---")
    
    if not os.path.exists(json_path):
        print(f"ERROR: Path file not found at {json_path}")
        return None, None
    
    try:
        with open(json_path, 'r') as f:
            path_data = json.load(f)
        
        if not isinstance(path_data, list) or len(path_data) == 0:
            print("ERROR: Invalid path data format")
            return None, None
        
        # Extract positions and look_at targets
        positions = []
        look_ats = []
        for entry in path_data:
            if 'pos' in entry:
                positions.append(entry['pos'])
            if 'look_at' in entry:
                look_ats.append(entry['look_at'])
        
        if len(positions) == 0:
            print("ERROR: No positions found in path data")
            return None, None
        
        trajectory = np.array(positions, dtype=np.float32)
        look_at_targets = np.array(look_ats, dtype=np.float32) if len(look_ats) == len(positions) else None
        
        print(f"Loaded {len(trajectory)} waypoints from JSON")
        
        # Remove consecutive duplicate positions that could cause singular matrices
        if len(trajectory) > 1:
            unique_mask = np.ones(len(trajectory), dtype=bool)
            for i in range(1, len(trajectory)):
                if np.allclose(trajectory[i], trajectory[i-1], atol=1e-6):
                    unique_mask[i] = False
            
            if not np.all(unique_mask):
                trajectory = trajectory[unique_mask]
                if look_at_targets is not None:
                    look_at_targets = look_at_targets[unique_mask]
                print(f"Removed {np.sum(~unique_mask)} duplicate positions. Final count: {len(trajectory)}")
        
        return trajectory, look_at_targets
        
    except Exception as e:
        print(f"ERROR loading path: {e}")
        return None, None

def interpolate_path(waypoints, look_at_targets=None, step_size=0.1):
    """
    Interpolate waypoints and look_at targets to create a smooth path with intermediate points.
    
    Args:
        waypoints: Nx3 array of waypoint positions
        look_at_targets: Nx3 array of look_at target positions (optional)
        step_size: Maximum distance between consecutive points in the output
    
    Returns:
        Interpolated positions and look_at targets (if provided)
    """
    if len(waypoints) < 2:
        return waypoints, look_at_targets
    
    print(f"\n--- Interpolating Path (step_size={step_size}) ---")
    interpolated_positions = []
    interpolated_look_ats = [] if look_at_targets is not None else None
    
    for i in range(len(waypoints) - 1):
        start_pos = waypoints[i]
        end_pos = waypoints[i + 1]
        
        # Calculate distance between waypoints
        distance = np.linalg.norm(end_pos - start_pos)
        
        # Calculate number of intermediate points needed
        num_steps = max(1, int(np.ceil(distance / step_size)))
        
        # Create interpolated points (excluding the end point to avoid duplicates)
        for j in range(num_steps):
            t = j / num_steps
            pos = start_pos * (1 - t) + end_pos * t
            interpolated_positions.append(pos)
            
            # Interpolate look_at targets as well
            if look_at_targets is not None:
                start_look = look_at_targets[i]
                end_look = look_at_targets[i + 1]
                look = start_look * (1 - t) + end_look * t
                interpolated_look_ats.append(look)
    
    # Add the final point
    interpolated_positions.append(waypoints[-1])
    if look_at_targets is not None:
        interpolated_look_ats.append(look_at_targets[-1])
    
    interpolated_path = np.array(interpolated_positions, dtype=np.float32)
    interpolated_look = np.array(interpolated_look_ats, dtype=np.float32) if interpolated_look_ats else None
    
    print(f"Interpolated {len(waypoints)} waypoints to {len(interpolated_path)} points")
    
    return interpolated_path, interpolated_look

def load_existing_path_pipeline(json_path=Config.JSON_PATH, interpolation_step=0.1):
    """
    Load an existing path from JSON and perform minimal scene setup for rendering.
    
    Args:
        json_path: Path to the JSON file containing waypoints
        interpolation_step: Step size for path interpolation (smaller = smoother)
    """
    global STAGE1_PCD, STAGE1_VOXEL_GRID, STAGE1_VOXEL_SIZE, STAGE1_SCENE_CENTER
    global STAGE4_FINAL_TRAJECTORY, STAGE4_TIMESTAMPS, STAGE4_LOOK_AT_TARGETS

    print("\n===========================================")
    print("   LOADING EXISTING PATH FROM JSON        ")
    print("===========================================")
    print_environment_info()
    pipeline_start_time = time.time()

    if o3d is None:
        print(">>> Pipeline aborted: Open3D is required."); return

    # --- STAGE 1: Minimal Geometry Analysis (needed for scene center and orientation) ---
    coords = load_and_filter_ply(Config.INPUT_PLY_PATH, Config.OPACITY_THRESHOLD)
    if len(coords) == 0: return

    pcd = create_and_clean_pcd(coords, Config.SOR_NB_NEIGHBORS, Config.SOR_STD_RATIO)
    if pcd is None: return

    STAGE1_PCD, STAGE1_SCENE_CENTER = normalize_scene(pcd)

    STAGE1_VOXEL_GRID, STAGE1_VOXEL_SIZE = adaptive_voxelization(STAGE1_PCD, Config.TARGET_VOXEL_RESOLUTION)
    if STAGE1_VOXEL_GRID is None: return

    # Analyze Scene Orientation
    detected_up_vector = analyze_scene_orientation(STAGE1_PCD, STAGE1_VOXEL_SIZE)
    Config.GLOBAL_UP_VECTOR = detected_up_vector

    # --- LOAD PATH FROM JSON ---
    loaded_trajectory, loaded_look_ats = load_path_from_json(json_path)
    if loaded_trajectory is None:
        print(">>> Failed to load path from JSON"); return
    
    if len(loaded_trajectory) < 2:
        print(">>> ERROR: Path must have at least 2 waypoints"); return
    
    # --- INTERPOLATE PATH ---
    if interpolation_step > 0:
        loaded_trajectory, loaded_look_ats = interpolate_path(loaded_trajectory, loaded_look_ats, step_size=interpolation_step)
    
    # The loaded path is already in world coordinates, so we need to normalize it
    # by subtracting the scene center (same as done in normalize_scene)
    normalized_trajectory = loaded_trajectory - STAGE1_SCENE_CENTER
    normalized_look_ats = loaded_look_ats - STAGE1_SCENE_CENTER if loaded_look_ats is not None else None
    
    # Validate that normalization didn't create degenerate positions
    print(f"\nPath statistics after normalization:")
    print(f"  Min position: {np.min(normalized_trajectory, axis=0)}")
    print(f"  Max position: {np.max(normalized_trajectory, axis=0)}")
    print(f"  Mean position: {np.mean(normalized_trajectory, axis=0)}")
    
    # Check for any remaining issues
    deltas = np.linalg.norm(np.diff(normalized_trajectory, axis=0), axis=1)
    min_delta = np.min(deltas)
    max_delta = np.max(deltas)
    print(f"  Minimum step size: {min_delta:.6f}")
    print(f"  Maximum step size: {max_delta:.6f}")
    
    if min_delta < 1e-6:
        print("WARNING: Very small steps detected in trajectory")
    
    STAGE4_FINAL_TRAJECTORY = normalized_trajectory
    STAGE4_LOOK_AT_TARGETS = normalized_look_ats
    
    # Create timestamps based on the trajectory length
    num_frames = len(STAGE4_FINAL_TRAJECTORY)
    duration = num_frames / Config.VIDEO_FPS
    STAGE4_TIMESTAMPS = np.linspace(0, duration, num_frames)
    
    print(f"\n>>> Path Loading Completed Successfully in {time.time() - pipeline_start_time:.2f} seconds.")
    print(f"Trajectory has {len(STAGE4_FINAL_TRAJECTORY)} waypoints")
    if STAGE4_LOOK_AT_TARGETS is not None:
        print(f"Look-at targets loaded: {len(STAGE4_LOOK_AT_TARGETS)} points")
    print(f"Video duration: {duration:.2f} seconds")

def run_rendering_pipeline(camera_roll_degrees=180):
    """
    Run the rendering pipeline with optional camera roll.
    
    Args:
        camera_roll_degrees: Roll angle in degrees (90 = rotate left, -90 = rotate right, 0 = no rotation)
    """
    global STAGE1_PCD, STAGE1_SCENE_CENTER
    global STAGE4_FINAL_TRAJECTORY, STAGE4_LOOK_AT_TARGETS
    global STAGE5_CAMERA_POSES_C2W_ORIGINAL

    print("\n===========================================")
    print("   RENDERING PIPELINE   ")
    print("===========================================")

    if STAGE4_FINAL_TRAJECTORY is None:
        print("ERROR: No trajectory found. Load path first."); return
    
    if len(STAGE4_FINAL_TRAJECTORY) < 2:
        print("ERROR: Trajectory must have at least 2 waypoints."); return

    # --- Calculate Orientations ---
    print(f"Using Detected Global Up Vector for Stabilization: {Config.GLOBAL_UP_VECTOR}")
    
    # Debug: Check trajectory validity before orientation calculation
    print(f"\nTrajectory validation:")
    print(f"  Number of waypoints: {len(STAGE4_FINAL_TRAJECTORY)}")
    print(f"  First waypoint: {STAGE4_FINAL_TRAJECTORY[0]}")
    print(f"  Last waypoint: {STAGE4_FINAL_TRAJECTORY[-1]}")
    if STAGE4_LOOK_AT_TARGETS is not None:
        print(f"  Look-at targets available: {len(STAGE4_LOOK_AT_TARGETS)} points")
        print(f"  First look-at: {STAGE4_LOOK_AT_TARGETS[0]}")
        print(f"  Last look-at: {STAGE4_LOOK_AT_TARGETS[-1]}")
    
    # Calculate orientations using look_at targets if available
    if STAGE4_LOOK_AT_TARGETS is not None:
        rotations = calculate_orientations_from_look_at(STAGE4_FINAL_TRAJECTORY, STAGE4_LOOK_AT_TARGETS, Config.GLOBAL_UP_VECTOR)
    else:
        rotations = calculate_base_orientations(STAGE4_FINAL_TRAJECTORY, Config.GLOBAL_UP_VECTOR)

    # Apply camera roll if specified
    if camera_roll_degrees != 0:
        rotations = apply_camera_roll(rotations, camera_roll_degrees)

    # Smooth Orientations
    smoothed_rotations = smooth_orientations_gimbal(rotations, Config.GIMBAL_SMOOTHING_SIGMA)
    
    # Assemble Poses
    Poses_Norm, STAGE5_CAMERA_POSES_C2W_ORIGINAL = assemble_camera_poses(
        STAGE4_FINAL_TRAJECTORY, smoothed_rotations, STAGE1_SCENE_CENTER
    )
    
    # Debug: Check pose validity
    print(f"\nPose validation:")
    print(f"  Number of poses: {len(STAGE5_CAMERA_POSES_C2W_ORIGINAL)}")
    first_pose = STAGE5_CAMERA_POSES_C2W_ORIGINAL[0]
    print(f"  First pose determinant: {np.linalg.det(first_pose[:3, :3]):.6f}")
    print(f"  First pose position: {first_pose[:3, 3]}")
    
    # Check if any poses are singular
    for i, pose in enumerate(STAGE5_CAMERA_POSES_C2W_ORIGINAL):
        det = np.linalg.det(pose[:3, :3])
        if abs(det) < 1e-6:
            print(f"WARNING: Pose {i} has near-zero determinant: {det}")
            if i < 5:
                print(f"  Pose {i}:\n{pose}")
    
    # Visualize Path Geometry
    visualize_final_path(
        STAGE1_PCD, STAGE4_FINAL_TRAJECTORY, Config.OUTPUT_DIR
    )
    
    # Render Final Video
    render_video(
        STAGE5_CAMERA_POSES_C2W_ORIGINAL, Config.INPUT_PLY_PATH, Config, Config.OUTPUT_DIR
    )
    
    print("\n=== PIPELINE COMPLETED ===")

# Execute the pipeline
if __name__ == "__main__":
    # Check if the input file exists before starting
    if not os.path.exists(Config.INPUT_PLY_PATH):
        print(f"CRITICAL ERROR: Input file not found at {Config.INPUT_PLY_PATH}")
        print("Please ensure the dataset is correctly linked in the environment.")
    else:
        # Load existing path from JSON with interpolation
        # interpolation_step: smaller values = smoother path (more frames)
        load_existing_path_pipeline(Config.JSON_PATH, interpolation_step=0.1)


   LOADING EXISTING PATH FROM JSON        
--- Environment Info ---
Compute Device: cuda:0
Open3D Tensor Device: CUDA:0
GSplat Available: True
------------------------

--- Step 1.1: Loading PLY file from /kaggle/input/ply-scenes/input-data/outdoor-drone_exported.ply ---
Loaded 3185436 points. Filtered to 2962617.
Step 1.1 completed in 0.16 seconds.

--- Step 1.2 & 1.3: Creating Open3D PCD and Cleaning (SOR) ---
Points remaining: 2944436
Step 1.2 & 1.3 completed in 10.96 seconds.

--- Step 1.4: Normalization (Centering) ---

--- Step 1.5: Adaptive Voxelization ---
Calculated adaptive voxel size: 0.4010
Step 1.5 completed in 0.11 seconds.

--- Step 1.6 (NEW): Analyzing Scene Orientation (Up-Vector) ---
Dominant vertical axis detected: Y
Successfully Detected Global Up-Vector: [0. 1. 0.]
Step 1.6 completed in 0.17 seconds.

--- Loading Path from /kaggle/input/drone-path-v3/planned_path.json ---
Loaded 57 waypoints from JSON
Removed 1 duplicate positions. Final count: 56

--- Interpolati

In [23]:
run_rendering_pipeline()


   RENDERING PIPELINE   
Using Detected Global Up Vector for Stabilization: [0. 1. 0.]

Trajectory validation:
  Number of waypoints: 586
  First waypoint: [-0.4807203   0.00924526  0.38874073]
  Last waypoint: [-2.57780291  0.00924526  7.075117  ]
  Look-at targets available: 586 points
  First look-at: [0.0192797  0.00924526 0.08874072]
  Last look-at: [0.0192797  0.00924526 0.08874072]

--- Step 5.1: Calculating Camera Orientations from Look-At Targets ---
Step 5.1 completed in 0.00 seconds.

--- Applying Camera Roll: 180 degrees ---

--- Step 5.2 Smoothing (Sigma=15.0) ---

--- 5.3 Assembling Poses ---

Pose validation:
  Number of poses: 586
  First pose determinant: -1.000000
  First pose position: [-0.5         0.          0.30000001]

--- 5.4 Saving Path Visualization ---
Saved ./output_results/path_vis.ply

--- Step 5.5: Rendering Video (GSplat Rasterization) ---
Loading full PLY data for rendering from /kaggle/input/ply-scenes/input-data/outdoor-drone_exported.ply...
Loaded 